In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime

2023-03-10 13:34:29.197661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# This turns all the axes white in all the matplotlib plots. Comment this out if you dont want that
COLOR = 'white'
matplotlib.rcParams['text.color'] = COLOR
matplotlib.rcParams['axes.labelcolor'] = COLOR
matplotlib.rcParams['xtick.color'] = COLOR
matplotlib.rcParams['ytick.color'] = COLOR

In [4]:
df_transaction = pd.read_csv('../datasets/ieee-fraud-detection/train_transaction.csv')

df_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# Features that are used, isFraud is the target
features = ['isFraud', 'TransactionDT',
            'TransactionAmt','ProductCD', 'card4', 'P_emaildomain','R_emaildomain']

In [44]:
df = df_transaction[features]
df.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card4,P_emaildomain,R_emaildomain
0,0,86400,68.5,W,discover,NaN,NaN
1,0,86401,29.0,W,mastercard,gmail.com,NaN
2,0,86469,59.0,W,visa,outlook.com,NaN
3,0,86499,50.0,W,mastercard,yahoo.com,NaN
4,0,86506,50.0,H,mastercard,gmail.com,NaN


In [45]:
# Sort features into categorical and numerical types
target = 'isFraud'
# Categorical features
cat = ['TransactionDT','ProductCD', 'card4', 'P_emaildomain','R_emaildomain']
# Numeric features
num = ['TransactionAmt']

In [46]:
# Drop rows with missing features
df = df.dropna()
y = df[target].values
df.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card4,P_emaildomain,R_emaildomain
10,0,86549,75.887,C,mastercard,gmail.com,gmail.com
11,0,86555,16.495,C,mastercard,hotmail.com,hotmail.com
40,0,87209,75.887,C,visa,gmail.com,gmail.com
48,0,87317,42.294,C,visa,outlook.com,outlook.com
49,0,87317,3.595,C,mastercard,anonymous.com,anonymous.com


In [47]:
x_cat = df.filter(items = cat).values
x_num = df.filter(items = num).values


In [48]:
labelencoder_X = LabelEncoder()
# Label encode every categorical column
for i in range(len(cat)): 
    x_cat[:, i] = labelencoder_X.fit_transform(x_cat[:, i])

In [ ]:
#ohencoder_X = OneHotEncoder()
# One-hot encode every categorical column
#for i in range(len(cat)): 
    #x_cat[:, i] = ohencoder_X.fit_transform(x_cat[:, i])

In [49]:
# Build input vector X, the training data
X = np.concatenate((x_cat, x_num), axis=1)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [51]:
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32').reshape((-1,1))

In [52]:
# Initialize model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
2521/2521 [==============================] - 4s 1ms/step - loss: 22.4880 - accuracy: 0.8553
Epoch 2/5
2521/2521 [==============================] - 3s 1ms/step - loss: 0.5039 - accuracy: 0.9105
Epoch 3/5
2521/2521 [==============================] - 3s 1ms/step - loss: 0.2881 - accuracy: 0.9163
Epoch 4/5
2521/2521 [==============================] - 3s 1ms/step - loss: 0.3146 - accuracy: 0.9161
Epoch 5/5
2521/2521 [==============================] - 3s 1ms/step - loss: 0.3153 - accuracy: 0.9163


In [53]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(val_loss, val_acc)

631/631 [==============================] - 1s 948us/step - loss: 0.2782 - accuracy: 0.9204
0.278199702501297 0.9204027056694031
